In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

from torch import nn
import torchvision
from torchvision.models import resnet50, efficientnet_b5, densenet161
from torchvision import transforms
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, roc_curve, RocCurveDisplay
# sensitivity, specificity, precision, recall, f1_score, roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix, classification_report, hamming_loss, zero_one_loss
from tqdm import tqdm
import wandb

import albumentations as A

In [2]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'mps'
model_path = "./AUG-New-IISc/best_model/models/best_model.pth"

In [3]:
ROOT = "./data-iisc"
images_dir = "train_data_dir"
train_csv = "train_data.csv"
val_csv = "val_data.csv"
test_csv = "test_data.csv"
class Dataset(BaseDataset):

    def __init__(
            self,
            images_dir,
            aug_fn=None,
            preprocessing=None,
    ):

        self.ids = [img for img in os.listdir(images_dir) if img.endswith('.png')]

        self.images = [os.path.join(images_dir, img) for img in self.ids]

        self.aug_fn = aug_fn
        self.preprocessing = preprocessing

    def __getitem__(self, i):
        image = cv2.imread(self.images[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.aug_fn:
            sample = self.aug_fn(image.shape)(image=image)
            image = sample['image']

        if self.preprocessing:
            sample = self.preprocessing(image=image)
            image = sample['image']

        return image, self.ids[i]
    
    def __len__(self):
        return len(self.images)

In [4]:
# get image, pass it through albumentations, get the predicted labels, and then show the image
def resize_image(image_shp, target_size=512):
    """
    Resize the image to the target size
    :param image: The image to resize
    :param target_size: The target size
    :return: The resized image
    """
    h, w, _ = image_shp

    max_size = max(h, w)

    transform = A.Compose([
    A.PadIfNeeded(min_height=max_size, min_width=max_size, border_mode=cv2.BORDER_CONSTANT, value=(255, 255, 255)),
    A.Resize(512, 512, interpolation=cv2.INTER_AREA),
    # A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    ])

    return transform

# %%
def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype("float32")


def get_preprocessing(preprocessing_fn=None):
    """Construct preprocessing transform

    Args:
        preprocessing_fn (callbale): data normalization function
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose

    """

    _transform = [
        # Lambda(image=preprocessing_fn),
        A.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return A.Compose(_transform)

In [ ]:
model = densenet161(weights='DEFAULT')
model.classifier = torch.nn.Linear(2208, 6)

model.to(DEVICE)

checkpoint = torch.load(model_path,map_location=DEVICE)
model.load_state_dict(checkpoint['model_state_dict'])


In [6]:
def inference_epoch_runner_confidence_output(description:str, loader, model, device="cuda"):

    header = ["ImageID", "Class 1","Class 2","Class 3","Class 4","Class 5","Class 6"]
    epoch_df = pd.DataFrame(columns=header)

    model.eval()
    
    with torch.no_grad():
        with tqdm(loader, desc=description.title()) as iterator:
            for images, img_ids in iterator:

                images = images.to(device)
                outputs = model.forward(images)
                sigmoid_outputs = torch.sigmoid(outputs)

                for i in range(len(sigmoid_outputs)):
                    # get the image id
                    # get the sigmoid outputs

                    img_id = img_ids[i]
                    sigmoid_output = np.round(sigmoid_outputs[i].cpu().numpy().astype("float32"),5)

                    # print(type(img_id), type(sigmoid_output), type(sigmoid_output.tolist()), type(list(sigmoid_output)))
                    epoch_df.loc[len(epoch_df)] = [img_id] +  list(sigmoid_output)


    return epoch_df

In [7]:
def inference_epoch_runner_2(description:str, loader, model, device="cuda"):
    labels = ["Class 1","Class 2","Class 3","Class 4","Class 5","Class 6"] # hidden class names
    header = ["ImageID"] + [f"CLASS {i}" for i in range(1,5)] + [f"PROB {i}" for i in range(1,5)]
    epoch_df = pd.DataFrame(columns=header)

    model.eval()
    
    with torch.no_grad():
        with tqdm(loader, desc=description.title()) as iterator:
            for images, img_ids in iterator:

                images = images.to(device)
                outputs = model.forward(images)
                sigmoid_outputs = torch.sigmoid(outputs)

                for i in range(len(sigmoid_outputs)):

                    top_4, top_4_indices = torch.topk(sigmoid_outputs[i], 4)
                    img_id = img_ids[i]

                    top_4 = list(np.round(top_4.cpu().numpy().astype("float32"),5))
                    top_4_indices = list(top_4_indices.cpu().numpy().astype("int8"))

                    epoch_df.loc[len(epoch_df)] = [img_id] +  [labels[i] for i in top_4_indices] + top_4


                # predicted = (torch.sigmoid(outputs) >= 0.5).int()
                # _, top_3 = torch.topk(torch.sigmoid(outputs), 3)
                # top_3_labels.extend(top_3.cpu().numpy().astype("int8"))
                # predicted_labels.extend(predicted.cpu().numpy().astype("int8"))

    return epoch_df

In [ ]:
dest_dir = "./Confidence-scores"
dirs = [
    ("./Results/Train Data","CITY_1_2_Train"),
    ("./Results/Test Images/CITY_1_2/Data_Classification","CITY_1_2_Test"),
    ("./Results/Test Images/CITY_3/Test_data_CITY_3_1","CITY_3_1_Test"),
    ("./Results/Test Images/CITY_3/Test_data_CITY_3_2","CITY_3_2_Test"),
    ("./Results/Test Images/CITY_3/Test_data_CITY_3_3","CITY_3_3_Test"),
    ("./Results/Test Images/CITY_3/Test_data_CITY_3_3a","CITY_3_3a_Test"),
    ("./Results/Test Images/CITY_4","CITY_4_Test"),
]

# Gives the confidence scores for each class
for item in dirs:
    inference_dataset = Dataset(
    images_dir=item[0],
    aug_fn=resize_image,
    preprocessing=get_preprocessing()
    )

    inference_loader = DataLoader(inference_dataset, batch_size=8, shuffle=False)

    epoch_df = inference_epoch_runner_confidence_output("Inference", inference_loader, model, device=DEVICE)
    epoch_df.to_csv(f"{dest_dir}/confidence_outputs/{item[1]}_confidence.csv", index=False)

# Gives the top 4 classes and their probabilities
for item in dirs:
    inference_dataset = Dataset(
    images_dir=item[0],
    aug_fn=resize_image,
    preprocessing=get_preprocessing()
    )

    inference_loader = DataLoader(inference_dataset, batch_size=8, shuffle=False)

    epoch_df = inference_epoch_runner_2("Inference", inference_loader, model, device=DEVICE)
    epoch_df.to_csv(f"{dest_dir}/top_4/{item[1]}_Top_4.csv", index=False)

    
